# Example of prepending to a zarr without having to re-write the chunks

In [1]:
from rolling_zarr_array_store import OffsetArrayStoreMixin, CHUNK_REGEX
import numpy as np
import zarr
import json

In [2]:
class Store(OffsetArrayStoreMixin, dict):
    pass

Create a store with the values 0-4, shape=(5,) chunks=(1,)

In [3]:
store = Store(cache_offset=False)

data = zarr.zeros(shape=(5,), store=store, chunks=[1])
data[:] = np.linspace(0,4,5)
data[:]

array([0., 1., 2., 3., 4.])

Take a look at how that chunk data is stored in the store

In [4]:
print("key\tvalue")
for k,v in ((k,v) for k,v in store.items() if CHUNK_REGEX.match(k)):
    print(f"{k}\t{data._decode_chunk(v)}")

key	value
0	[0.]
1	[1.]
2	[2.]
3	[3.]
4	[4.]


Now we add an offset of -1. This effectivly shifts access such that axising index `1` gives `0`, `2` -> `1` etc allowing us to redefine what accessing `0` will give. We also resize the array as we are adding data. 

In [5]:
offset = [-1]
store['.zattrs'] = json.dumps({'_offset': offset})

val_to_prepend = np.array([-1.0])
store['0'] = data._encode_chunk(val_to_prepend)
data.resize(data.shape[0] + 1)


Reopen the store as a zarr as our original zarr array has likely cached the data.

In [6]:
prepended = zarr.open(store)

We have successfully prepended some data

In [7]:
prepended[:]

array([-1.,  0.,  1.,  2.,  3.,  4.])

Note the original data hasn't moved in the store:

In [8]:
print("key\tvalue")
for k,v in ((k,v) for k,v in store.items() if CHUNK_REGEX.match(k)):
    print(f"{k}\t{data._decode_chunk(v)}")

key	value
0	[0.]
1	[1.]
2	[2.]
3	[3.]
4	[4.]
-1	[-1.]
